In [1]:
import os
import sqlite3

In [2]:
folder = 'Downloads/data_wrangling'
folder_size = 0
for (path, dirs, files) in os.walk(folder):
    for file in files:
        if str(file[0]) != '.':
            filename = os.path.join(path, file)
            folder_size = os.path.getsize(filename)
            print(file, " = %0.1f MB" % (folder_size/(1024*1024.0)))

hcm.db  = 43.4 MB
hcm.osm  = 84.1 MB
sample.osm  = 1.2 MB
nodes.csv  = 32.0 MB
nodes_tags.csv  = 3.6 MB
ways.csv  = 2.7 MB
ways_nodes.csv  = 10.6 MB
ways_tags.csv  = 3.4 MB


In [5]:
db='Downloads/data_wrangling/hcm.db'

In [6]:
con = sqlite3.connect(db)
cur = con.cursor()

In [7]:
node = cur.execute('SELECT COUNT(*) FROM nodes')
node.fetchone()[0]

404079

In [6]:
ways = cur.execute('SELECT COUNT(*) FROM ways')
ways.fetchone()[0]

47878

In [7]:
unique_user = cur.execute('SELECT COUNT(DISTINCT(sq.uid)) \
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) as sq')
unique_user.fetchone()[0]

1596

In [8]:
top_10_active = cur.execute('SELECT sq.user, COUNT(*) as num \
            FROM  (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as sq\
            GROUP BY sq.user \
            ORDER BY num DESC \
            LIMIT 10')
top_10_active.fetchall()

[('TuanIfan', 126416),
 ('Ivan Garcia', 74020),
 ('Dymo12', 38772),
 ('QuangDBui@TMA', 27482),
 ('KimChinhTri', 23062),
 ('528491', 18767),
 ('guenter', 10013),
 ('tandung', 9106),
 ('mrjoba', 7725),
 ('dotnam', 6090)]

In [9]:
user_once_only = cur.execute('SELECT count(*)\
            FROM (SELECT sq.user, COUNT(*) as num \
            FROM  (SELECT user FROM nodes UNION ALL SELECT user FROM ways) as sq\
            GROUP BY sq.user \
            HAVING num = 1) as sq2')
user_once_only.fetchone()[0]

713

In [10]:
#Additional improvement, phone number inconsistency
phone=cur.execute('SELECT sq.value\
            FROM (SELECT value FROM nodes_tags\
            WHERE key="phone"\
            UNION ALL\
            SELECT value FROM ways_tags \
            WHERE key="phone") as sq\
            LIMIT 20')
phone.fetchall()

[('0839307627',),
 ('+84 8 39991612 ',),
 ('8222373',),
 ('8368761',),
 ('8361679',),
 ('8365073',),
 ('39206472',),
 ('38365868',),
 ('38369123',),
 ('38379589',),
 ('+84 838369268 / +84 838361935',),
 ('38375582',),
 ('+84838370447',),
 ('3829 2772',),
 ('+084 0917 685 464',),
 ('+8438231130',),
 ('01252026241',),
 ('+84 8 38272372',),
 ('+84838389523',),
 ('84839105689',)]

In [8]:
con.close()